Which k-mers are contributing to the matches? Realistically, I want to be able to show this for every protein match

# Imports and setup

### Auto-re-import python modules, useful for editing local files

In [2]:
%load_ext autoreload
%autoreload 2

## Imports

In [3]:
import os

import polars as pl
import rich
import screed

# Handwritten local modules
from sig2kmer import degenerate_protein_chatgpt
from sigseq import SigSeq

import sourmash

# Read in data

In [4]:
ksize = 10
moltype = "hp"

In [5]:
cleaned_multisearch_folder = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/analysis-outputs/hp/00_cleaned_multisearch_results"

pq = os.path.join(
    cleaned_multisearch_folder, f"scope40.multisearch.{moltype}.k{ksize}.filtered.pq"
)
multisearch = pl.scan_parquet(pq)
multisearch

In [6]:
multisearch.filter(pl.col("query_scop_id") == "d1ty4a_")

### Filter on query family ["Family f.1.4.1: Bcl-2 inhibitors of programmed cell death"](https://scop.berkeley.edu/sunid=56855)

In [ ]:
bcl2_family = "f.1.4.1"

multisearch_bcl2 = multisearch.filter((pl.col("query_family") == bcl2_family)).collect()
multisearch_bcl2

In [7]:
# apoptosis = multisearch.filter(pl.col("query_name").str.contains("A|apoptosis"))
# apoptosis

In [8]:
sig_folder = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/sigs"

sigfile = os.path.join(
    sig_folder,
    f"query.astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.{moltype}.k{ksize}.sig.zip",
)

sigs = {x.name.split()[0]: x for x in sourmash.load_file_as_signatures(sigfile)}
len(sigs)

15177

In [9]:
query_scop_id_index = multisearch.columns.index("query_scop_id")
match_scop_id_index = multisearch.columns.index("match_scop_id")

for row in multisearch_bcl2.iter_rows():
    query_id = row[query_scop_id_index]
    match_id = row[match_scop_id_index]
    query_sig = sigs[query_id]
    match_sig = sigs[match_id]
    # overlap = query_sig.
    break

In [10]:
# query_sig.minhash.intersection(match_sig.minhash)

In [11]:
def overlap(sig1, sig2):
    """
    provide detailed comparison of two signatures

    Cribbed from: https://github.com/sourmash-bio/sourmash/blob/2cc44e026bc147a292a1dcac3ab2bcd046be09fa/src/sourmash/sig/__main__.py#L379C1-L457C1
    """

    try:
        similarity = sig1.similarity(sig2)
    except ValueError:
        raise

    cont1 = sig1.contained_by(sig2)
    cont2 = sig2.contained_by(sig1)

    name1 = sig1.name
    name2 = sig2.name

    md5_1 = sig1.md5sum()
    md5_2 = sig2.md5sum()

    ksize = sig1.minhash.ksize
    moltype = sig1.minhash.moltype

    num = sig1.minhash.num
    size1 = len(sig1.minhash)
    size2 = len(sig2.minhash)

    scaled = sig1.minhash.scaled

    hashes_1 = set(sig1.minhash.hashes)
    hashes_2 = set(sig2.minhash.hashes)

    num_common = len(hashes_1 & hashes_2)
    disjoint_1 = len(hashes_1 - hashes_2)
    disjoint_2 = len(hashes_2 - hashes_1)
    num_union = len(hashes_1.union(hashes_2))

    print(
        """\
first signature:
  signature: {name1}
  md5: {md5_1}
  k={ksize} molecule={moltype} num={num} scaled={scaled}

second signature:
  signature: {name2}
  md5: {md5_2}
  k={ksize} molecule={moltype} num={num} scaled={scaled}

similarity:                  {similarity:.5f}
first contained in second:   {cont1:.5f}
second contained in first:   {cont2:.5f}

number of hashes in first:   {size1}
number of hashes in second:  {size2}

number of hashes in common:  {num_common}
only in first:               {disjoint_1}
only in second:              {disjoint_2}
total (union):               {num_union}
""".format(
            **locals()
        )
    )


overlap(query_sig, match_sig)

first signature:
  signature: d1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}
  md5: 5f9e6eb4503a7471cfc700032d52e90e
  k=27 molecule=hp num=0 scaled=1

second signature:
  signature: d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}
  md5: e0f4b134db68bb57a20d17b9ac232e91
  k=27 molecule=hp num=0 scaled=1

similarity:                  0.02498
first contained in second:   0.04000
second contained in first:   0.03846

number of hashes in first:   75
number of hashes in second:  78

number of hashes in common:  3
only in first:               72
only in second:              75
total (union):               150



### Read fasta sequences

In [12]:
fasta = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/rawdata/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.fa"

sequences = {}

with screed.open(fasta) as records:
    for record in records:
        name = record["name"].split()[0]
        sequences[name] = record["sequence"].upper()
len(sequences)

15177

In [13]:
# ! head $fasta

In [14]:
match_id

'd3gcea_'

In [15]:
query_sig.minhash.moltype

'hp'

In [16]:
query_seq = sequences[query_id]
match_seq = sequences[match_id]
# list(query_sig.minhash.kmers_and_hashes(match_seq, is_protein=True))

In [28]:
query_sig.name

'd1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}'

In [29]:
match_sig.name

'd3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}'

In [26]:
query_seq

'TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK'

In [27]:
match_seq

'STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK'

In [18]:
# sequence_kmers_in_sig(match_sig, query_seq)

In [19]:
query_seq

'TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK'

In [20]:
degenerate_protein_chatgpt(query_seq, "hp")

'phphhhphhhphhhhphhphhhphpphhhhhhphhhhphphhhhhphpphphphhhhhphphphphphphhpphhpphhhphpppphpphphhpphhhppp'

In [21]:
match_seq

'STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK'

In [22]:
degenerate_protein_chatgpt(match_seq, "hp")

'pphhphhphpphphhhhphhphphpphhhhpphpphhhhpphppphphhhphhhhphpphphpphhphhphphpphhhphhhphhhhphhphhhphpphhhhhp'

In [23]:
len(query_seq)

101

In [24]:
query_sig.name

'd1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}'

In [25]:
match_sig.name

'd3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}'

## Write class and code to show overlap between two sequences

In [37]:
query = SigSeq(query_sig, query_seq)
match = SigSeq(match_sig, match_seq)

query.display_alignment(match)

---

d1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 
143350]}

Match from 3-32

                                                                        TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAG
LLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK

                                                                        phphhhphhhphhhhphhphhhphpphhhhhhphhhhphphhh
hhphpphphphhhhhphphphphphphhpphhpphhhphpppphpphphhpphhhppp

|||||||||||||||||||||||||||||

pphhphhphpphphhhhphhphphpphhhhpphpphhhhpphppphphhhphhhhphpphphpphhphhphphpphhhphhhphhhhphhphhhphpphhhhhp

STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK

Match from 75-104

d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}

## ITerate over some rows

In [ ]:
query_scop_id_index = multisearch.columns.index("query_scop_id")
match_scop_id_index = multisearch.columns.index("match_scop_id")

for row in multisearch.head(10).iter_rows():
    query_id = row[query_scop_id_index]
    match_id = row[match_scop_id_index]

    query = SigSeq(sigs[query_id], sequences[query_id])
    match = SigSeq(sigs[match_id], sequences[match_id])

    query.display_alignment(match)

## Iterate over Multisearch BCL2

In [ ]:
query_scop_id_index = multisearch.columns.index("query_scop_id")
match_scop_id_index = multisearch.columns.index("match_scop_id")

for row in multisearch_bcl2.iter_rows():
    query_id = row[query_scop_id_index]
    match_id = row[match_scop_id_index]

    query = SigSeq(sigs[query_id], sequences[query_id])
    match = SigSeq(sigs[match_id], sequences[match_id])

    query.display_alignment(match)